# Funções Auxiliares e Imports

## Imports

In [70]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import random as random
from scipy.stats import pearsonr
from scipy import stats
from scipy.linalg import expm
import pandas as pd
import seaborn as sns

file_path = "C:/Users/gabri/Downloads/Redes Complexas/"

## Lendo e plotando o grafo dos dados

### Base USairport500

In [71]:
# Base de dados para a questão 1
usa = nx.read_edgelist(file_path + "data/USairport500.txt")

# plt.figure(figsize=(12,10))
# pos = nx.spring_layout(usa)
# nx.draw(usa, pos, node_color="lightgray", node_size=50, with_labels=True)

### Base Hamsterster

In [72]:
# Base de dados para a questão 2
ham = nx.read_edgelist(file_path + "data/hamsterster.txt")

# plt.figure(figsize=(12,10))
# pos = nx.spring_layout(ham)
# nx.draw(ham, pos, node_color="lightgray", node_size=500, with_labels=True)

### Base Jazz

In [73]:
# Base de dados para a questão 3
jazz = nx.read_edgelist(file_path + "data/jazz.txt")

# plt.figure(figsize=(12,10))
# pos = nx.spring_layout(adv)
# nx.draw(adv, pos, node_color="lightgray", node_size=500, with_labels=True)

### Base Word_Adjacencies

In [74]:
# Base de dados para as questões 4 e 5
word = nx.read_weighted_edgelist(file_path + "data/word_adjacencies.txt")

# plt.figure(figsize=(12,10))
# pos = nx.spring_layout(word)
# nx.draw(word, pos, node_color="lightgray", node_size=500, with_labels=True)

### Padronização da Rede

In [75]:
# Função que "padroniza" a rede, transformando-a em uma rede sem direção, e retornando o maior componente conectado para a análise.
def padronizar_rede(G):
    # Transformamos o grafo em uma rede sem direção.
    G = G.to_undirected()
    G.remove_edges_from(nx.selfloop_edges(G))

    # Vamos selecionar apenas o maior componente conectado.
    Gcc = sorted(nx.connected_components(G), key=len, reverse=True)
    G = G.subgraph(Gcc[0])

    # Transformando os labels para números inteiros, começando com 0:

    G = nx.convert_node_labels_to_integers(G, first_label=0)

    # Número de vértices e arestas:

    N = len(G)
    M = G.number_of_edges()
    print('Number of nodes:', N)
    print('Number of edges:', M)
    return G



## Funções do Questionário 1

### Cálculo do Grau Médio

In [76]:
# Função para calcular o grau médio do grafo.
def mean_degree(G):
    vk = dict(G.degree()).values()
    vk = np.array(list(vk))
    print('Degree', vk)



    md = np.mean(vk)
    # print('Mean degree: ', md)
    return md



### Cálculo da Distribuição do Grau

In [77]:
# Função para calcular a distribuição do grau do grafo.
def degree_distribution(G):
    vk = dict(G.degree())
    vk = list(vk.values())  # we get only the degree values
    vk = np.array(vk)
    maxk = np.max(vk)
    mink = np.min(vk)
    kvalues= np.arange(0,maxk+1) # possible values of k
    Pk = np.zeros(maxk+1) # P(k)
    for k in vk:
        Pk[k] = Pk[k] + 1
    Pk = Pk/sum(Pk) # the sum of the elements of P(k) must to be equal to one
    return kvalues,Pk

# Função para calcular o momento da distribuição do grau do grafo.
def momment_of_degree_distribution(G,m):
    k,Pk = degree_distribution(G)
    M = sum((k**m)*Pk)
    return M

# Função para calcular a variância do grau.
def variance(G):
    variance = momment_of_degree_distribution(G,2) - momment_of_degree_distribution(G,1)**2
    return variance
# print("Variância do grau = ", variance)

### Cálculo do Momento do Grau

In [78]:
# Função para calcular o momento do grafo.
def momment(G,m):
    M = 0
    N = len(G)
    for i in G.nodes:
        M = M + G.degree(i)**m
    M = M/N
    return M

### Cálculo da Entropia de Shannon

In [79]:
# Função para calcular a entropia de Shannon do grafo.
def shannon_entropy(G):
    k,Pk = degree_distribution(G)
    H = 0
    for p in Pk:
        if(p > 0):
            H = H - p*np.math.log(p, 2)
    return H

# Função para calcular a entropia de Shannon normalizada do grafo.
def normalized_shannon_entropy(G):
    k,Pk = degree_distribution(G)
    H = 0
    for p in Pk:
        if(p > 0):
            H = H - p*np.math.log(p, 2)
    return H/np.math.log(len(G),2)

### Cálculo da Transitividade e do Coeficiente de Agrupamento

In [80]:
# Função para calcular o coeficiente de transitividade do grafo.
def graph_transitivity(G):
    CC = (nx.transitivity(G))
    # print("Transitivity = ","%3.4f"%CC)
    return "%3.4f"%CC

# Função para calcular o coeficiente de agrupamento do grafo.
def avg_clustering_coefficient(G):
    avc = nx.average_clustering(G)
    # print("Average clustering:", "%3.4f"%avc)
    return "%3.4f"%avc

## Funções do Questionário 2

### Distâncias

In [81]:
# Função para calcular e plotar a distribuição de grau da rede.
def graph_avg_distance(G):

    if nx.is_connected(G) == True:
        l = nx.average_shortest_path_length(G)
        print("Average shortest path length:", "%3.4f"%l)
    else:
        print("The graph has more than one connected component")
    return "%3.4f"%l
    

# Função para calcular o diâmetro do grafo.
def graph_diameter(G):
    d = nx.diameter(G)
    print('Network diameter:', d)
    return d

# Função para calcular a eficiência global e local do grafo.
def graph_efficiency(G):

    E = nx.global_efficiency(G)
    print('Network efficiency', E)

    leff = nx.local_efficiency(G)
    print('The average local efficiency of the network:', leff)

# Função para calcular a distância geodésica do grafo.
def graph_geodesic_distance(G):
    N = len(G)
    d = graph_diameter(G)

    if nx.is_connected(G) == True:
        D = np.zeros(shape=(N,N)) # D is the matrix of distances
        vl = []
        for i in np.arange(0,N):
            for j in np.arange(i+1, N):
                if(i != j):
                    aux = nx.shortest_path(G,i,j)
                    dij = len(aux)-1
                    D[i][j] = dij
                    D[j][i] = dij
                    vl.append(dij)
        x = range(0,d+1)
        plt.hist(vl, bins = x, density=True, color='#0504aa',alpha=0.7, rwidth=0.85)
        plt.title("Distribution of the geodesic distances", fontsize=20)
        plt.ylabel("P(l)", fontsize=15)
        plt.xlabel("Shortest path length (l)", fontsize=15)
        #plt.grid(True)
        plt.savefig('av_short_path.svg')
        plt.show(True)
    else:
        print("The graph has more than one connected component")



### Cálculo da Correlação de Grau a Grau

In [82]:
# Função para calcular e plotar a correlação de grau a grau (degree-degree correlation).
def degree_correlation(G):
    r=nx.degree_assortativity_coefficient(G)
    print("Assortativity = ","%3.4f"%r)

# Função para calcular a correlação de Pearson entre os graus dos nós.
def pearson_correlation(G):
    ki = []
    kj = []
    for i in range(0,len(G.nodes())):
        for j in range(0, len(G.nodes())):
            if(G.has_edge(i,j) == True):
                ki.append(G.degree(i))
                kj.append(G.degree(j))
                
    
    # calculate Pearson's correlation
    corr, _ = pearsonr(ki, kj)
    print('Pearsons correlation: %.3f' % corr)

# Função para calcular o grau médio de vizinhança (knn) do grafo.
def avg_neighbor_degree(G):

    knn = []
    for i in G.nodes():
        aux =  nx.average_neighbor_degree(G, nodes = [i])
        knn.append(float(aux[i]))
    knn = np.array(knn)
    print("Average degree of the neighborhood of the network:", "%3.2f"%np.mean(knn))
    return knn


# Função para calcular a correlação entre conectividade média da vizinhança
#  e grau do nó. Positiva em redes sortidas, negativa em redes assortativas.
def avc_assnet_degree(G):
    knn = avg_neighbor_degree(G)

    vk = dict(G.degree())
    vk = list(vk.values())



    knnk = list()
    ks = list()
    for k in np.arange(np.min(vk), np.max(vk)+1):
        aux = vk == k
        if(len(knn[aux]) > 0):
            av_knn = np.mean(knn[aux]) #average clustering among all the nodes with degree k
            knnk.append(av_knn)
            ks.append(k)
    fig= plt.figure(figsize=(10,6))

    plt.plot(ks, knnk, '-o', color='gray',markersize=10, linewidth=0,
            markerfacecolor='lightgray',
            markeredgecolor='black',
            markeredgewidth=2)
    #plt.loglog(ks,knnk,'bo',basex=10,basey=10)
    #plt.title("Average neighborhood connectivity vs degree")
    plt.ylabel("knn(k)", fontsize = 20)
    plt.xlabel("k", fontsize = 20)
    #plt.savefig('knnk.eps')

    # determine best fit line
    par = np.polyfit(ks, knnk, 1, full=True)
    slope=par[0][0]
    intercept=par[0][1]
    xl = [min(ks), max(ks)]
    yl = [slope*xx + intercept  for xx in xl]
    plt.plot(xl, yl, '--', linewidth=3, color='black')
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.savefig('knn.eps') #save the figure into a file
    plt.show(True)

    return ks, knnk

# Função para calcular o coeficiente de correlação de Pearson entre
#  o grau do nó e a conectividade média da vizinhança.
def pearson_correlation_knn(ks, knnk):
    rho = np.corrcoef(ks, knnk)[0,1]
    print('Pearson correlation coefficient:', rho)

# Função para calcular o coeficiente de correlação de Spearman entre
#  o grau do nó e a conectividade média da vizinhança.
def spearman_correlation_knn(ks, knnk):
    s = stats.spearmanr(ks, knnk)
    print('Spearman rank correlation coefficient:', s[0])
    print('p-value:', s[1])

### Caminhadas Aleatórias

In [83]:
# Função para realizar caminhadas aleatórias no grafo
def random_walk(G, T, seed_node):
    
    # neighbors
    ng = G.neighbors(seed_node)
    walk = []
    for t in range(0,T):
        next_node = random.choice(list(ng))
        ng = G.neighbors(next_node)
        walk.append(next_node)
    print('Walk:', walk)



    nx.draw(G, with_labels = True, node_size=500, font_size=16, pos = nx.spring_layout(G))
    plt.show(True)

    A = nx.to_numpy_array(G)
    print(A)
    print('Number of walks of length two:')
    print(A@A)



## Funções do Questionário 3

### Funções de Centralidades

In [84]:
# Função para calcular a centralidade de grau do grafo.
# Degree centrality: C(i) = k(i)/(N-1)
def degree_centrality(G):
    vk = dict(G.degree())
    vk = list(vk.values())
    print('Degree centrality', vk)
    return vk

# Função para plotar a distribuição da centralidade de grau do grafo.
def degree_centrality_distribution(vk):
    plt.figure(figsize=(6,4))
    plt.hist(vk, density=True)
    plt.title("Distribution of the Degree Centrality", fontsize=20)
    plt.ylabel("P(k)", fontsize=20)
    plt.xlabel("Degree Centrality (k)", fontsize=20)
    #plt.grid(True)
    plt.savefig('degree-centrality.eps')
    plt.show(True)

# Função para calcular a centralidade de proximidade do grafo.
# Closeness centrality: C(i) = 1/(sum_{j=1}^{N} d(i,j))
def closeness_centrality(G):
    clc = dict(nx.closeness_centrality(G))
    print('Closeness centrality', clc)

    clc = list(clc.values())
    av_clc = np.mean(clc)
    print('Average closeness centrality', av_clc)

    return clc, av_clc

# Função para plotar a distribuição da centralidade de proximidade do grafo.
def closeness_centrality_distribution(clc):
    # Distribuição de Closeness Centrality
    plt.figure(figsize=(6,4))
    plt.hist(clc, density=True)
    plt.title("Distribution of the Closeness Centrality", fontsize=20)
    plt.ylabel("P(CLC)", fontsize=20)
    plt.xlabel("Closeness Centrality (CLC)", fontsize=20)
    plt.savefig('closeness.eps')
    plt.show(True)

    
# Função para calcular a centralidade de intermediação do grafo.
# Betweenness centrality: C(i) = sum_{s!=i!=t} (sigma(s,t|i)/sigma(s,t))
def betweenness_centrality(G):
    btc = dict(nx.betweenness_centrality(G))
    print('Betweenness centrality', btc)

    btc = list(btc.values())
    av_btc = np.mean(btc)
    print('Average betweenness centrality', av_btc)

    return btc, av_btc

# Função para plotar a distribuição da centralidade de intermediação do grafo.
def betweenness_centrality_distribution(btc):
    
    # Distribuição de Betweenness Centrality
    plt.figure(figsize=(6,4))
    plt.hist(btc, density=True)
    plt.title("Distribution of the Betweenness Centrality", fontsize=20)
    plt.ylabel("P(BTC)", fontsize=20)
    plt.xlabel("Betweenness Centrality (BTC)", fontsize=20)
    plt.savefig('betweenness.eps')
    plt.show(True)

    
# Função para calcular a centralidade de autovetor do grafo.
# Eigenvector centrality: C(i) = sum_{j=1}^{N} (A(i,j)*C(j))
def eigenvector_centrality(G):
    evec = dict(nx.eigenvector_centrality(G))
    print('Eigenvector centrality', evec)

    evec = list(evec.values())
    av_evec = np.mean(evec)
    print('Average eigenvector centrality', av_evec)

    return evec, av_evec
    
# Função para plotar a distribuição da centralidade de autovetor do grafo.
# A distribuição é feita em relação ao valor absoluto da centralidade de autovetor.
def eigenvector_centrality_distribution(evec):
    # Distribuição de Eigenvector Centrality
    plt.figure(figsize=(6,4))
    plt.hist(evec, density=True)
    plt.title("Distribution of the Eigenvector Centrality", fontsize=20)
    plt.ylabel("P(EVC)", fontsize=20)
    plt.xlabel("Eigenvector Centrality (EVC)", fontsize=20)
    plt.savefig('eigenvector.eps')
    plt.show(True)

    
# Função para calcular a centralidade de PageRank do grafo.
# PageRank centrality: C(i) = (1-d)/N + d*sum_{j=1}^{N} (A(i,j)*C(j)/k(j))
def pagerank(G):
    pr = dict(nx.pagerank(G, alpha=0.85))
    print('PageRank: ', pr)

    pr = list(pr.values())
    pr = np.array(pr)
    av_pr = np.mean(pr)
    print('Average PageRank centrality', av_pr)

    B = dict(nx.betweenness_centrality(G))
    B = list(B.values())
    B = np.array(B)
    plt.figure(figsize=(6,4))
    plt.ylabel("Betweennees centrality", fontsize = 20)
    plt.xlabel("Page Rank", fontsize = 20)
    plt.plot(pr, B, 'ro')
    plt.show(True)

    return pr, av_pr

# Função para plotar a distribuição da centralidade de PageRank do grafo.
# A distribuição é feita em relação ao valor absoluto da centralidade de PageRank.
def pagerank_distribution(pr):
    # Distribuição de PageRank Centrality
    plt.figure(figsize=(6,4))
    plt.hist(pr, density=True)
    plt.title("Distribution of the PageRank Centrality", fontsize=20)
    plt.ylabel("P(PRC)", fontsize=20)
    plt.xlabel("PageRank Centrality (PRC)", fontsize=20)
    plt.savefig('pagerank.eps')
    plt.show(True)

    
# Função para calcular a centralidade de Katz do grafo.
# Katz centrality: 
# C(i) = sum_{j=1}^{N} (A(i,j)*C(j)) + alpha*sum_{j=1}^{N} (A(i,j)*C(j))
def k_core(G):
    KC = dict(nx.core_number(G))
    print('K-core: ', KC)
    KC = list(KC.values())
    KC = np.array(KC)

    return KC

# Função para plotar a distribuição da centralidade de K-core do grafo.
# A distribuição é feita em relação ao valor absoluto da centralidade de K-core.
def k_core_distribution(KC):
    # Distribuição de K-core
    plt.figure(figsize=(6,4))
    plt.hist(KC, density=True)
    plt.title("Distribution of the K-core", fontsize=20)
    plt.ylabel("P(KC)", fontsize=20)
    plt.xlabel("K-core (KC)", fontsize=20)
    plt.savefig('k-core.eps')
    plt.show(True)

    
# Função para plotar a rede.
def plotting_graph(G, node_size, node_color, with_labels):
    plt.figure(figsize=(12,10))
    pos = nx.spring_layout(G)
    nx.draw(G, pos, node_color=node_color, 
            node_size=node_size, with_labels=with_labels)
    plt.show(True)


### Acessibilidade de Caminhadas Aleatórias

In [85]:
# Função para calcular a acessibilidade do grafo.
def acc(G):
    N = len(G.nodes())
    vk = dict(G.degree())
    vk = list(vk.values())
    A = nx.adjacency_matrix(G)
    P = np.zeros((N,N), dtype = 'float')
    for i in np.arange(0, N):
        for j in np.arange(0, N):
            if(vk[i] > 0):
                P[i,j] = A[i,j]/vk[i]
    P2 = expm(P)/np.exp(1)
    vacc = np.zeros(N, dtype = float)
    for i in np.arange(0, N):
        acc = 0
        for j in np.arange(0,N):
            if(P2[i,j] > 0):
                acc = acc + P2[i,j]*np.log(P2[i,j])
        acc = np.exp(-acc)
        vacc[i] = acc
    
    print('ACC: ', vacc)


    plt.figure(figsize=(6,6))
    pos=nx.spring_layout(G)
    nx.draw(G, with_labels = True, pos = pos)
    plt.show(True)  

    return vacc


# Função para calcular a acessibilidade do grafo de acordo com o 
# modelo de Hub and Leaves.
def hub_acc(N):
    x = np.cosh(1)/np.exp(1)
    y = np.sinh(1)/np.exp(1)
    acchub = np.exp(-x*np.log(x) -y*np.log(y/(N-1)))
    print('Hub acc:', acchub)
    # Accessibility of the leaves
    x = np.sinh(1)/np.exp(1)
    y = (np.cosh(1)-1)/(np.exp(1)*(N-1))
    accleaves = np.exp(-x*np.log(x) - (N-2)*y*np.log(y) \
                       - (1/np.exp(1) + y)*np.log(1/np.exp(1) + y))
    print('Leaves acc:', accleaves)

    return acchub, accleaves

### Correlação entre medidas de centralidade

In [86]:
# Função para calcular a matriz de correlações entre as centralidades.
def centralities_df(G):
    vk = degree_centrality(G)
    clc, av_clc = closeness_centrality(G)
    btc, av_btc = betweenness_centrality(G)
    evec, av_evec = eigenvector_centrality(G)
    pr, av_pr = pagerank(G)
    KC = k_core(G)
    df = pd.dataframe({'Degree': vk, 'Closeness': clc, 'Betweenness': btc, \
                    'Eigenvector': evec, 'PageRank': pr, 'K-core': KC})
    
    return df

# Função para plotar a matriz de correlação entre as centralidades.
def correlation_matrix(df):
    corr = df.corr()
    #Plot Correlation Matrix using Matplotlib
    plt.figure(figsize=(8,8))
    plt.imshow(corr, cmap='Blues', interpolation='none', aspect='auto')
    plt.colorbar()
    plt.xticks(range(len(corr)), corr.columns, rotation='vertical', fontsize=20)
    plt.yticks(range(len(corr)), corr.columns, fontsize=20)
    plt.suptitle('Correlation between centrality measures', fontsize=20)
    plt.grid(False)
    plt.show()

# Função para plotar a matriz de correlação entre as centralidades usando o Seaborn.
def correlogram(df):
    sns.pairplot(df)
    plt.show(True)

# Função para plotar o mapa de calor da matriz de correlação entre as centralidades.
def heatmap(df):
    corr = df.corr()
    plt.figure(figsize=(7, 7))
    mask = np.zeros_like(corr)
    mask[np.triu_indices_from(mask)] = True
    with sns.axes_style("white"):ax = sns.heatmap(corr, mask=mask, vmax=1, square=True)
    plt.show()

# Questões

## Questão 1)

A média medida eigenvector centrality da rede USairport500 é igual a quanto? Considere apenas o maior componente. Use:


G = G.to_undirected()

G.remove_edges_from(nx.selfloop_edges(G))

Gcc = sorted(nx.connected_components(G), key=len, reverse=True)

G = G.subgraph(Gcc[0])

G = nx.convert_node_labels_to_integers(G, first_label=0)

In [87]:
# Padronizo a rede para a análise
usa = padronizar_rede(usa)

# Calculo e salvo a eigenvector centrality
evec, av_evec = eigenvector_centrality(usa)

Number of nodes: 500
Number of edges: 2980
Eigenvector centrality {0: 0.19076801166001284, 1: 0.03636566857696688, 2: 0.0691626602921969, 3: 0.003968107284167804, 4: 0.008793199264018935, 5: 0.07118060019092086, 6: 0.04546430744771243, 7: 0.07323642015910903, 8: 0.01730443424706561, 9: 0.017922732745642947, 10: 0.0897698767865774, 11: 0.07380066147981246, 12: 0.02184958926877229, 13: 0.12467071731482425, 14: 0.131436508778258, 15: 0.003968107284167804, 16: 0.023602572915143526, 17: 0.07362620372650132, 18: 0.13953024621380883, 19: 0.04641141422804443, 20: 0.02448577438054315, 21: 0.01743753895063849, 22: 0.02329810285937023, 23: 0.04561503424842579, 24: 0.13802628925967486, 25: 0.14913393470874747, 26: 0.09676493405122416, 27: 0.046025725596912495, 28: 0.012354640796703582, 29: 0.03949654624376495, 30: 0.007070203451269203, 31: 0.17727093382439202, 32: 0.016769651517526653, 33: 0.020450615709312264, 34: 0.06667976179297104, 35: 0.1307198987514137, 36: 0.15277202028115405, 37: 0.1782209

## Questão 2)

Calcule a correlação de Pearson entre a medida betweeness centrality e grau para a rede hamsterster

In [88]:
# Padronizo a rede para a análise
ham = padronizar_rede(ham)

# Calculo e salvo a betweness centrality
btc, av_btc = betweenness_centrality(ham)

vk = degree_centrality(ham)

# Calculo e ploto a correlação de Pearson entre a betweness centrality
# e o grau do nó
pearson_correlation_knn(vk, btc)

Number of nodes: 1788
Number of edges: 12476
Betweenness centrality {0: 0.0032301395159124173, 1: 0.004359163634432046, 2: 4.69760984669588e-06, 3: 0.0019246756055983022, 4: 0.0015897108730727908, 5: 0.008317832920302798, 6: 0.00042458389219936216, 7: 1.9738944899129653e-05, 8: 0.023830285305123603, 9: 0.011152942491924013, 10: 0.004368348881463037, 11: 0.0034727280151145012, 12: 0.005163274835100675, 13: 0.00411887790194941, 14: 0.02975972399325523, 15: 0.05758339013331758, 16: 0.0002996185250389925, 17: 0.0009208796235972814, 18: 0.01340604707192026, 19: 0.0, 20: 0.004584667123603669, 21: 0.005518916467675661, 22: 0.00024335317825260338, 23: 7.846059838832014e-06, 24: 0.009177311193294593, 25: 0.0015066002983463687, 26: 0.0015066002983463687, 27: 0.0035460458888329164, 28: 0.008128163481218163, 29: 0.01456782086301145, 30: 0.01794811494908073, 31: 0.03539561691905463, 32: 0.0018450219333241019, 33: 0.006158468825870166, 34: 0.0010998920747460827, 35: 0.01315125260071138, 36: 0.010443

## Questão 3)

Calcule o coeficiente de correlação de Spearman entre as medidas closeness centrality e k-core para a rede "Jazz"

In [89]:
# Padronizo a rede para a análise
jazz = padronizar_rede(jazz)

# Calculo e salvo a closeness centrality
clc, av_clc = closeness_centrality(jazz)

# Calculo e salvo a k-core centrality
KC = k_core(jazz)

# Calculo e ploto a correlação de Spearman entre a closeness centrality
# e a k-core centrality

spearman_correlation_knn(clc, KC)

Number of nodes: 198
Number of edges: 2742
Closeness centrality {0: 0.42919389978213507, 1: 0.4613583138173302, 2: 0.48402948402948404, 3: 0.4613583138173302, 4: 0.44269662921348313, 5: 0.4310722100656455, 6: 0.5898203592814372, 7: 0.42919389978213507, 8: 0.48284313725490197, 9: 0.49873417721518987, 10: 0.5038363171355499, 11: 0.5116883116883116, 12: 0.5367847411444142, 13: 0.5157068062827225, 14: 0.5157068062827225, 15: 0.4457013574660634, 16: 0.4129979035639413, 17: 0.5324324324324324, 18: 0.5253333333333333, 19: 0.5549295774647888, 20: 0.4613583138173302, 21: 0.4121338912133891, 22: 0.5811209439528023, 23: 0.5077319587628866, 24: 0.3588342440801457, 25: 0.4746987951807229, 26: 0.5225464190981433, 27: 0.5184210526315789, 28: 0.3724007561436673, 29: 0.37595419847328243, 30: 0.5225464190981433, 31: 0.5116883116883116, 32: 0.49747474747474746, 33: 0.3971774193548387, 34: 0.5090439276485789, 35: 0.4164904862579281, 36: 0.4497716894977169, 37: 0.4613583138173302, 38: 0.40286298568507156, 

## Questão 4) 

Escolha as medidas de centralidade.

- 1. É uma medida de centralidade local, pois alguns vértices com valores altos dessa medida podem estar na borda da rede.
	
Resposta: Degree.

A centralidade de grau é uma medida local, que simplesmente conta quantas conexões diretas um nó tem. Um nó pode estar na borda da rede, mas ainda assim ter um grau alto.


- 2. Um vértice é considerado central se estiver conectado a outros centrais. É usada em redes dirigidas.
	
Resposta: PageRank.

O PageRank funciona com a ideia de que conexões de nós importantes aumentam a importância do nó conectado.

- 3. Considera o menores caminhos entre os pares de vértices. 
	
Resposta: Closeness.

A centralidade de closeness mede o quão perto um nó está de todos os outros, com base nos menores caminhos.

- 4. É calculada a partir do espectro da matriz de adjacência.
	
Resposta: Eigenvector

A centralidade de autovetor (eigenvector) é baseada nos autovalores e autovetores da matriz de adjacência. Ela mede a influência de um nó, considerando a influência dos vizinhos.

- 5. É calculada a partir da exponencial da matriz de probabilidade de transição.
	
Resposta: Random walk

A centralidade por passeio aleatório (random walk) está ligada à matriz de transição (Markov), e muitas vezes usa sua exponencial para representar probabilidades ao longo do tempo.

- 6. Usa os menores caminhos de modo a determinar a carga (em termos da quantidade de tráfego) em cada vértice.

Resposta: Betweeness

A centralidade de intermediação (betweenness) mede o número de vezes que um nó aparece nos menores caminhos entre todos os pares